In [10]:
!cat ./mlir_tensor_lib.spy

from mlir import MLIR_Type, MLIR_op, MLIR_asm


# Define dynamically sized 1D tensor in MLIR
@blue.generic
def MLIR_tensor_1d(dtype: MLIR_Type):
    return MLIR_Type("tensor<?x{}>", dtype)

# Define dynamically sized 1D memref in MLIR
@blue.generic
def MLIR_memref_1d(dtype: MLIR_Type):
    return MLIR_Type("memref<?x{}>", dtype)


# Define a Pythonic wrapper to define the binary operations
@blue.generic
def make_tensor_type(DTYPE: MLIR_Type):
    # Define the lower level MLIR type of the tensor
    T = MLIR_tensor_1d[DTYPE]

    T_index = MLIR_Type("index")
    # MLIR_asm are directly inline MLIR asm.
    mlir_zero_index = MLIR_asm("arith.constant {value=0:index}", T_index, ())
    mlir_tensor_dim = MLIR_asm("tensor.dim", T_index, (T, T_index))
    mlir_tensor_empty = MLIR_asm("tensor.empty", T, (T_index,))
    # linalg.add is more complicated as it has regions
    # so using MLIR_op uses the MLIR python binding to fill in all the necessary
    # attributes.
    mlir_linalg_add = MLIR_

Compile the SPy file

In [11]:
stdout = !python -m nbcc.compiler -shared ./mlir_tensor_lib.spy mlir_tensor_lib.so

Disassemble the compiled file.

Note: the implementation is SIMD-vectorized.

In [12]:
!llvm-objdump -d mlir_tensor_lib.so


mlir_tensor_lib.so:	file format mach-o arm64

Disassembly of section __TEXT,__text:

00000000000003b8 <_spy_mlir_tensor_lib$export_tensor_f64_add>:
     3b8: a9bd57f6     	stp	x22, x21, [sp, #-0x30]!
     3bc: a9014ff4     	stp	x20, x19, [sp, #0x10]
     3c0: a9027bfd     	stp	x29, x30, [sp, #0x20]
     3c4: aa0603f4     	mov	x20, x6
     3c8: aa0303f5     	mov	x21, x3
     3cc: aa0103f3     	mov	x19, x1
     3d0: d37df068     	lsl	x8, x3, #3
     3d4: 91010100     	add	x0, x8, #0x40
     3d8: 9400013a     	bl	0x8c0 <_malloc+0x8c0>
     3dc: 9100fc08     	add	x8, x0, #0x3f
     3e0: 927ae501     	and	x1, x8, #0xffffffffffffffc0
     3e4: 937ffea8     	asr	x8, x21, #63
     3e8: ca150109     	eor	x9, x8, x21
     3ec: 91001d2a     	add	x10, x9, #0x7
     3f0: f100013f     	cmp	x9, #0x0
     3f4: 9a89b149     	csel	x9, x10, x9, lt
     3f8: ca890d09     	eor	x9, x8, x9, asr #3
     3fc: d37df128     	lsl	x8, x9, #3
     400: f100053f     	cmp	x9, #0x1
     404: 5400024b     	b.lt	0x44c 

DLopen the the compiled library to run the function here

In [ ]:
import ctypes

lib = ctypes.CDLL('./mlir_tensor_lib.so')
# tensor_add = getattr(lib, "_mlir_ciface_spy_mlir_tensor_lib$export_tensor_f64_add")
tensor_arrayexpr = getattr(lib, "_mlir_ciface_spy_mlir_tensor_lib$export_tensor_f64_arrayexpr")

In [4]:
from ctypes import CDLL, c_double, byref
from mlir.runtime import make_nd_memref_descriptor, get_ranked_memref_descriptor, ranked_memref_to_numpy
import numpy as np


In [5]:

NELEM = 200

memref_1d_f64 = make_nd_memref_descriptor(1, c_double)

A = np.arange(NELEM, dtype=np.float64) / NELEM
B = np.arange(NELEM, dtype=np.float64) / NELEM
C = np.arange(NELEM, dtype=np.float64) / NELEM

In [7]:

argA = get_ranked_memref_descriptor(A)
argB = get_ranked_memref_descriptor(B)
argC = get_ranked_memref_descriptor(C)

out_memref = (memref_1d_f64 * 1)()
args = [out_memref, byref(argA), byref(argB), byref(argC)]
tensor_arrayexpr(*args)

output = ranked_memref_to_numpy(out_memref)

np.testing.assert_allclose(output, (A + B) * (C + B))


In [8]:
%timeit (A + B) * (C + B)

974 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
out_memref = (memref_1d_f64 * 1)()
args = [out_memref, byref(argA), byref(argB), byref(argC)]

%timeit tensor_arrayexpr(*args)

350 ns ± 3.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
